In [14]:
import cv2
import numpy as np  ## **
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [45]:
cap = cv2.VideoCapture('vtest.avi')

# params for ShiTomasi corner detection
feature_params = dict(maxCorners=200, qualityLevel=0.1, minDistance=7, blockSize=7)

# params for calcOpticalFlowPyrLK
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.02))

# height, width, and fps of the video
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)

tracks = []   #
track_len = 15  # maximum number of corners in a track
frame_idx = 0   # frame counter
detect_interval = 5  # corner detection, every 5 frames

# Distance diff threshold value to filter out the "still" corner points
dist_diff = 0.28

while(True):
    ret, frame = cap.read()

    if ret:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        vis = frame.copy()

        # detect corners in each detect_interval
        if frame_idx % detect_interval==0:
            mask = np.zeros_like(frame_gray)
            mask[:] = 255

            if frame_idx !=0:
                for x,y in [np.int32(tr[-1]) for tr in tracks]:
                    cv2.circle(mask, (x, y), 5, 0, -1)

            p = cv2.goodFeaturesToTrack(frame_gray, mask=mask, **feature_params)

            if p is not None:
                for x, y in np.float32(p).reshape(-1,2):
                    tracks.append([(x, y)])
        
        # start optical flow, if the current frame is not the first frame and there are good features detected
        if frame_idx != 0 and len(tracks)>0:
            img0 ,img1 = prev_gray, frame_gray
            p0 = np.float32([tr[-1] for tr in tracks]).reshape(-1,1,2)
            
            # forward-tracking for new location of those corners (which were detected in the previous frame) in the current frame
            p1, st, err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)

            
            # back-tracking for match verification between frames
            p0r, _, _ = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)

            # check difference between the original locations and the back-tracked locations of the corners
            d = abs(p0-p0r).reshape(-1,2).max(-1)

            # if greater than 1, unstable corners
            good = d < 1

            new_tracks = []

            for i, (tr, (x, y), flag) in enumerate(zip(tracks, p1.reshape(-1, 2), good)):
                
                # check if unstable corner. if so, ignore
                if not flag:
                    continue

                # save the locations of a corners in the past and current frames
                tr.append((x, y))
                

                # only keep "track_len" number of locations of a corner
                if len(tr) > track_len:
                    del tr[0]

                # Calculate the difference between the prev and the cur corners                 
                prev_corner_index = len(tr) - 2
                cur_corner = tr[len(tr) - 1]
                prev_corner = tr[prev_corner_index]
                x_diff = abs(cur_corner[0] - prev_corner[0])
                y_diff = abs(cur_corner[1] - prev_corner[1])
                
                # only add and show the corner that has moved
                if x_diff > dist_diff or y_diff > dist_diff:
                    cv2.circle(vis, (int(x), int(y)), 2, (0, 255, 0), -1)
                    new_tracks.append(tr)

            # update tracks    
            tracks = new_tracks
            
            # draw trajectory              
            cv2.polylines(vis, [np.int32(tr) for tr in tracks], False, (0, 255, 0), 1)

        frame_idx += 1
        prev_gray = frame_gray

        cv2.imshow('track', vis)
        k = cv2.waitKey(30)
        if (k & 0xFF) == ord('q'):
            break
        if (k & 0xFF) == ord('p'):
            k2 = cv2.waitKey(-1)
            if (k2 & 0xFF) == ord('q'):
                break
    else:
        break

cv2.destroyAllWindows()
cap.release()